In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, TensorBoard, LambdaCallback
from keras.applications.resnet50 import ResNet50, preprocess_input 
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model
import random
import cv2
from PIL import Image
import os
from keras.utils import to_categorical
from collections import defaultdict
from PIL import Image
from io import BytesIO
import jpeg4py as jpeg

Using TensorFlow backend.


In [3]:
#parameters
classes = 10
batch_size= 100
image_size = 224
train_total = 3600
validation_total = 875

In [4]:
def CACNN_basic_block(x):
    x = Conv2D(8, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(16, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(32, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(64, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(128, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = GlobalAveragePooling2D()(x)
    return x

x = Input((image_size,image_size,3))

x1 = Conv2D(4, (3,3), padding='valid')(x)
x1 = BatchNormalization()(x1)
x1 = CACNN_basic_block(x1)

x2 = Conv2D(4, (5,5), padding='valid')(x)
x2 = BatchNormalization()(x2)
x2 = CACNN_basic_block(x2)

x3 = Conv2D(4, (7,7), padding='valid')(x)
x3 = BatchNormalization()(x3)
x3 = CACNN_basic_block(x3)

y = layers.concatenate([x1,x2,x3])
#y = Dropout(0.3)(y)
y = Dense(384, activation='relu')(y)
#y = Dropout(0.5)(y)
y = BatchNormalization()(y)
y = Dense(256, activation='relu')(y)
#y = Dropout(0.5)(y)
y = BatchNormalization()(y)
y = Dense(128, activation='relu')(y)
#y = Dropout(0.5)(y)
y = BatchNormalization()(y)
y = Dense(classes, activation='softmax')(y)

model  = Model(inputs=x,outputs=y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 222, 222, 4)  112         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 220, 220, 4)  304         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 218, 218, 4)  592         input_1[0][0]                    
__________________________________________________________________________________________________
batch_norm

In [5]:
train_datagen = ImageDataGenerator(horizontal_flip=True)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
                    

In [6]:
train_generator = train_datagen.flow_from_directory(
                    'train/',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )

validation_generator = validation_datagen.flow_from_directory(
                        'validation/',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)


Found 906500 images belonging to 10 classes.
Found 875 images belonging to 10 classes.


In [7]:
def image_aug_valid(image, index):
    Image.LOAD_TRUNCATED_IMAGES = True
    mpimg.LOAD_TRUNCATED_IMAGES = True
    if(index==0):
        res_image = np.uint8(cv2.pow(image/255.,0.8)*255.)
    elif(index==1):
        res_image = np.uint8(cv2.pow(image/255.,1.2)*255.)
    elif(index==2):
        res_image = cv2.resize(image,(0,0),fx=0.5,fy=0.5,interpolation = cv2.INTER_CUBIC)
    elif(index==3):
        res_image = cv2.resize(image,(0,0),fx=0.8,fy=0.8,interpolation = cv2.INTER_CUBIC)    
    elif(index==4):
        res_image = cv2.resize(image,(0,0),fx=1.5,fy=1.5,interpolation = cv2.INTER_CUBIC)
    elif(index==5):
        res_image = cv2.resize(image,(0,0),fx=2.0,fy=2.0,interpolation = cv2.INTER_CUBIC)
    elif(index==6):
        img = Image.fromarray(image)
        out_70_valid = BytesIO()
        img.save(out_70_valid, "JPEG", quality=70)
        res_image = jpeg.JPEG(np.frombuffer(out_70_valid.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_70_valid
    elif(index==7):
        img = Image.fromarray(image)
        out_90_valid = BytesIO()
        img.save(out_90_valid, "JPEG", quality=90)
        res_image = jpeg.JPEG(np.frombuffer(out_90_valid.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_90_valid
    return res_image 

In [8]:
def image_aug(image, index):
    Image.LOAD_TRUNCATED_IMAGES = True
    mpimg.LOAD_TRUNCATED_IMAGES = True
    if(index==0):
        res_image = np.uint8(cv2.pow(image/255.,0.8)*255.)
    elif(index==1):
        res_image = np.uint8(cv2.pow(image/255.,1.2)*255.)
    elif(index==2):
        res_image = cv2.resize(image,(0,0),fx=0.5,fy=0.5,interpolation = cv2.INTER_CUBIC)
    elif(index==3):
        res_image = cv2.resize(image,(0,0),fx=0.8,fy=0.8,interpolation = cv2.INTER_CUBIC)    
    elif(index==4):
        res_image = cv2.resize(image,(0,0),fx=1.5,fy=1.5,interpolation = cv2.INTER_CUBIC)
    elif(index==5):
        res_image = cv2.resize(image,(0,0),fx=2.0,fy=2.0,interpolation = cv2.INTER_CUBIC)
    elif(index==6):
        img = Image.fromarray(image)
        out_70 = BytesIO()
        img.save(out_70, "JPEG", quality=70)
        res_image = jpeg.JPEG(np.frombuffer(out_70.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_70
    elif(index==7):
        img = Image.fromarray(image)
        out_90 = BytesIO()
        img.save(out_90, "JPEG", quality=90)
        res_image = jpeg.JPEG(np.frombuffer(out_90.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_90
    return res_image 

In [9]:
def random_crop(image, crop_size):
    h,w,d = image.shape
    rand_num_h = random.randint(0,h-crop_size)
    rand_num_w = random.randint(0,w-crop_size)
    image_crop = image[rand_num_h:rand_num_h+crop_size,rand_num_w:rand_num_w+crop_size,:]
    return image_crop

In [10]:
def train_gen(train_files, train_classes, batch_size, target_size, imggen):
    min_batch_size = batch_size //10
    train_data = list(zip(train_files, train_classes))
    train_data_dict = defaultdict(list)
    for i in range(len(train_data)):
        train_data_dict[train_data[i][0].split('/')[0]].append(train_data[i])
       
    while(True):
        for i in train_data_dict.keys():
            random.shuffle(train_data_dict[i])
        for start in range(0, len(train_data), min_batch_size):
            image_crop_list = []
            image_classes_list = []
            
            
            for j in train_data_dict.keys():
                end = min(start + min_batch_size, len(train_data_dict[j])) 
                
                x_batch = [train_data_dict[j][i][0] for i in range(start,end)]
                
                for index,i in enumerate(x_batch):
                    rand_num = random.randint(0,7)
                    if(rand_num == 2 or rand_num == 3):
                        temp_file = random.choice(os.listdir('train_1024/'+i.split('/')[0]))
                        image = jpeg.JPEG('train_1024/' + i.split('/')[0]+'/' + temp_file).decode()
                    else:
                        image = jpeg.JPEG('train/' + i).decode()
                    
                    if(len(image.shape)<3):
                        print(_1_)
                        continue
                    
                    
                    image = image_aug(image,rand_num)
                    image = random_crop(image,image_size)
                    image_crop_list.append(image)
                    image_classes_list.append(train_data_dict[j][start+index][1])
                    
                    del image
                    
            
            if(len(image_classes_list)<batch_size-2*min_batch_size):
                break
            image_crop_list = [x/1. for x in image_crop_list]
            image_crop_list = [imggen.random_transform(x) for x in image_crop_list]
            image_crop_list = [preprocess_input(x) for x in image_crop_list]

            x_batch = np.array(image_crop_list, np.float32)
            y_batch = np.array(image_classes_list)         
            
            yield (x_batch, y_batch)

In [11]:
train_crop_generator = train_gen(train_generator.filenames,to_categorical(train_generator.classes),batch_size,image_size,train_datagen) 

In [12]:
def valid_gen(valid_files, valid_classes, batch_size, target_size, imggen):

    valid_data = list(zip(valid_files, valid_classes))
  
    while(True):
        
        for manip_num in range(8):
            for start in range(0, len(valid_data), batch_size):
                image_crop_list = []
                image_classes_list = []
                end = min(start + batch_size, len(valid_data)) 

                x_batch = [valid_data[i][0] for i in range(start,end)]

                for index,i in enumerate(x_batch):
                    if(manip_num == 2 or manip_num == 3):
                        temp_file = random.choice(os.listdir('validation_1024/'+i.split('/')[0]))
                        image = jpeg.JPEG('validation_1024/' + i.split('/')[0]+'/' + temp_file).decode()
                    else:
                        image = jpeg.JPEG('validation/' + i).decode()
                    if(len(image.shape)<3):
                            continue
                    #rand_num = random.randint(0,7)
                    
                    image = image_aug_valid(image,manip_num)
                    
                    image = random_crop(image,image_size)
                    image_crop_list.append(image)
                    image_classes_list.append(valid_data[start+index][1])

                    del image
                   


                image_crop_list = [x/1. for x in image_crop_list]
                image_crop_list = [preprocess_input(x) for x in image_crop_list]

                x_batch = np.array(image_crop_list, np.float32)
                y_batch = np.array(image_classes_list)         

                yield (x_batch, y_batch)

In [13]:
valid_aug_generator = valid_gen(validation_generator.filenames,to_categorical(validation_generator.classes),batch_size,image_size,validation_datagen) 

In [14]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=1e-4),
             metrics=['accuracy'])

In [15]:
from pushbullet import Pushbullet
pb = Pushbullet('o.KiDDDXPuzV4qKbXh4Lywbgw1tK2oFfq1')

In [16]:
pushbullet_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: pb.push_note("epoch: "+str(epoch),"train_loss: "+str(logs['loss'])+"    val_loss"+str(logs['val_loss'])))

In [17]:
callbacks = [ModelCheckpoint(filepath='cacnn_image_aug_1.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1,epsilon=0.01),
EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./4-metrics_cacnn_image_aug_1.csv'),
TensorBoard(log_dir='logs_cacnn_image_aug', write_graph=True),
            pushbullet_callback]

In [ ]:
history = model.fit_generator(
                    train_crop_generator,
                    steps_per_epoch = 8*int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=valid_aug_generator,
                    validation_steps= 8*int(np.ceil(validation_total/batch_size)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/100
287/288 [============================>.] - ETA: 1s - loss: 1.5874 - acc: 0.4769
Epoch 00001: val_loss improved from inf to 1.18234, saving model to cacnn_image_aug_1.hdf5
288/288 [==============================] - 456s 2s/step - loss: 1.5857 - acc: 0.4774 - val_loss: 1.1823 - val_acc: 0.6206
Epoch 2/100
287/288 [============================>.] - ETA: 1s - loss: 1.1637 - acc: 0.6161
Epoch 00002: val_loss improved from 1.18234 to 1.05759, saving model to cacnn_image_aug_1.hdf5
288/288 [==============================] - 415s 1s/step - loss: 1.1637 - acc: 0.6161 - val_loss: 1.0576 - val_acc: 0.6547
Epoch 3/100
254/288 [=========================>....] - ETA: 40s - loss: 1.0842 - acc: 0.6422

In [18]:
model.load_weights('cacnn_2_image_aug_1.hdf5')

In [ ]:
model.evaluate_generator(valid_aug_generator,8*int(np.ceil(validation_total/batch_size)))

[2.383350227560316, 0.0572857144820903]

In [33]:
# validation predictions for analysis

In [34]:
predictions_valid = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))
predictions_valid = np.argmax(predictions_valid, axis=1)

In [35]:
validation_generator.class_indices

{'HTC-1-M7': 0,
 'LG-Nexus-5x': 1,
 'Motorola-Droid-Maxx': 2,
 'Motorola-Nexus-6': 3,
 'Motorola-X': 4,
 'Samsung-Galaxy-Note3': 5,
 'Samsung-Galaxy-S4': 6,
 'Sony-NEX-7': 7,
 'iPhone-4s': 8,
 'iPhone-6': 9}

In [36]:
true_positive = np.zeros(10)
false_positive = np.zeros(10)
true_negative = np.zeros(10)
false_negative = np.zeros(10)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [37]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)


true_positive
[ 86.  72.  53.  53.  62.  64.  82.  68.  68.  81.]
false_positive
[ 16.  27.   5.  19.  31.  22.  12.   7.   8.  39.]
false_negative
[ 12.  27.  22.  24.  24.  12.  17.  18.  20.  10.]
Total
[ 98.  99.  75.  77.  86.  76.  99.  86.  88.  91.]


In [1]:
#test predictions - 512 size

In [22]:
filenames_list = []
images_list = []
for i in os.listdir('test/temp/'):
    img = Image.open('test/temp/'+i)
    filenames_list.append(i)
    images_list.append(np.array(img, np.float32))    

In [23]:
images_list = [preprocess_input(x) for x in images_list]

In [24]:
new_array_arr = np.array(images_list)

In [25]:
new_array_arr.shape

(2640, 512, 512, 3)

In [26]:
test_preds = model.predict(new_array_arr)

In [28]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}

In [52]:
predicted_classes = [class_ids[np.argmax(i)] for i in test_preds]

In [58]:
for index,i in enumerate(filenames_list):
    if(i.split('_')[-1]=='manip.tif'):
        predicted_classes[index]=''

In [60]:
submission = pd.DataFrame({'fname':filenames_list,'camera':predicted_classes})
submission.to_csv('submission_resnet_with_aug_trained_fully_512_only_unalt.csv', encoding="utf8", index=False)

In [61]:
from IPython.display import FileLink
FileLink('submission_resnet_with_aug_trained_fully_512_only_unalt.csv')

/home/janardhan/camera_model/submission_resnet_with_aug_trained_fully_512_only_unalt.csv